# Geometric Brownian Motion

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")
using LaTeXStrings

In [ ]:
function solution(S_0::Float64, σ::Float64)
    F_ana(t1, t2) = S_0^2 * exp(mu * (t1 + t2)) * (exp(σ^2 * min(t1, t2)) - 1);
    
    function fv!(out, _, t1, t2)
      out[1] = 0
      out[2] = mu * F[t1, t2]
    end
    function fd!(out, _, t1, t2)
      out[1] = mu * S[t1, t2]
      out[2] = 2mu * F[t1, t2] + σ^2 * (S[t1, t2]^2 + F[t1, t2])
    end
  
    S = GreenFunction(S_0 * ones(1,1), Symmetrical)
    F = GreenFunction(0.0 * ones(1,1), Symmetrical)
    
    sol = kbsolve!(fv!, fd!, [S, F], (0.0, T), atol=1e-9, rtol=1e-7, dtini=1e-10)
         
    return [sol.t, S.data, F.data, [F_ana(t1, t2) for t1 in sol.t, t2 in sol.t]]
end;

In [ ]:
T = 1.0
mu = 1.
t_scale = (mu == 0. ? 1. : abs(mu))

S_0 = [1., 2., 5.]
sigma = [1., .5, .1]

s = mapreduce(solution, hcat, S_0, sigma);

## Plotting

In [ ]:
cmap = "gist_heat";
colors = ["C0", "C1", "C2"];
lss = ["-", "--", "-."];

In [ ]:
figure(figsize=(7, 3))

ax = subplot(121)
for k in eachindex(sigma)
    if k == 1
        plot(s[1,k], diag(s[3,k]), label=L"F(t, t)", lw=1.5, c=colors[k], ls=lss[k])
        plot(s[1,k], diag(s[2,k]), label=L"\bar{S}(t)", lw=3.5, c=colors[k], ls=lss[k])
    else
        plot(s[1,k], diag(s[3,k]), lw=1.5, c=colors[k], ls=lss[k])
        plot(s[1,k], diag(s[2,k]), lw=3.5, c=colors[k], ls=lss[k])
    end
end
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, 15)
ax.set_xlabel(L"\mu t")
ax.legend(loc="best", handlelength=1.5, frameon=false, borderpad=0, labelspacing=0.25, fontsize="small")
ax.get_legend().legendHandles[1].set_color("k")
ax.get_legend().legendHandles[2].set_color("k")

ax = subplot(122)
semilogy([], [], label=L"\sigma / \mu", c="w")
for k in eachindex(sigma)
    semilogy(s[1,k], abs.((diag(s[3,k])  .- diag(s[4,k]))), lw=1.5, c=colors[k], label=L"%$(string(sigma[k]/(mu)))", ls=lss[k])
end

ax.set_xlim(0, t_scale * T)
ax.set_ylim(1e-9, 1e-5)
ax.set_xlabel(L"\mu t")
ax.set_ylabel(L"\left|F(t, t) - \mathcal{F}(t, t)\right|", labelpad=10)

# ax.yaxis.set_ticks_position("right")
ax.yaxis.set_label_position("right")

ax.legend(loc="best", frameon=false, labelspacing=0.0, borderpad=0, handlelength=1.5, fontsize="small")

tight_layout(pad=0.25, w_pad=1, h_pad=0)

savefig("geometric_brownian_motion_example_1.pdf")

In [ ]:
function meshgrid(xin,yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny,nx)
  yout=zeros(ny,nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix,jx]=xin[jx]
          yout[ix,jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

In [ ]:
Y, X = meshgrid(s[1,1], s[1,1]);

In [ ]:
figure(figsize=(7, 3))

vmin = 0
# vmax = D/(2lambda)
ax = subplot(121) # plt.gca()
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, s[3,1], cmap=cmap, rasterized=true)#, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\lambda t\$")
ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])

ax = subplot(122)
heatmap = ax.pcolormesh(t_scale * X, t_scale * Y, (abs.(s[3,1] .- s[4,1])), cmap="gist_gray", rasterized=true)#, vmin=1e-9, vmax=1e-6)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$\\lambda t\$")
# ax.set_ylabel("\$\\lambda t'\$")
ax.set_xlim(0, t_scale * T)
# ax.set_ylim(0, t_scale * T)
ax.set_xticks(t_scale .* [0, T/2, T])
ax.set_yticks(t_scale .* [0, T/2, T])
ax.set_yticklabels([])

tight_layout(pad=0.75, w_pad=0.25, h_pad=0)
# savefig("geometric_brownian_motion_example_2.pdf")